## Imports & constants

In [140]:
import os
from typing import List
import json
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil
import sys
import logging 

logging.basicConfig(
     level=logging.INFO, 
     format= '[%(asctime)s|%(levelname)s|%(module)s.py:%(lineno)s] %(message)s',
     datefmt='%H:%M:%S'
 )
import tqdm.notebook as tq
from tqdm import tqdm
# Create new `pandas` methods which use `tqdm` progress
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()

from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

import torch
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, 
    TrainingArguments, Trainer, EarlyStoppingCallback, IntervalStrategy
)

from defi_textmine_2025.data import TARGET_COL, INTERIM_DIR, MODELS_DIR

In [138]:
RANDOM_SEED = 123  # random reproducibility
np.random.seed(RANDOM_SEED)
# BASE_CHECKPOINT = "bert-base-uncased"
BASE_CHECKPOINT = "bert-base-multilingual-cased"
# BASE_CHECKPOINT = "camembert/camembert-base"
TASK_NAME = "hasrelation"
TASK_TARGET_COL = "label" # hasrelation?

entity_classes = {'TERRORIST_OR_CRIMINAL', 'LASTNAME', 'LENGTH', 'NATURAL_CAUSES_DEATH', 'COLOR', 'STRIKE', 'DRUG_OPERATION', 'HEIGHT', 'INTERGOVERNMENTAL_ORGANISATION', 'TRAFFICKING', 'NON_MILITARY_GOVERNMENT_ORGANISATION', 'TIME_MIN', 'DEMONSTRATION', 'TIME_EXACT', 'FIRE', 'QUANTITY_MIN', 'MATERIEL', 'GATHERING', 'PLACE', 'CRIMINAL_ARREST', 'CBRN_EVENT', 'ECONOMICAL_CRISIS', 'ACCIDENT', 'LONGITUDE', 'BOMBING', 'MATERIAL_REFERENCE', 'WIDTH', 'FIRSTNAME', 'MILITARY_ORGANISATION', 'CIVILIAN', 'QUANTITY_MAX', 'CATEGORY', 'POLITICAL_VIOLENCE', 'EPIDEMIC', 'TIME_MAX', 'TIME_FUZZY', 'NATURAL_EVENT', 'SUICIDE', 'CIVIL_WAR_OUTBREAK', 'POLLUTION', 'ILLEGAL_CIVIL_DEMONSTRATION', 'NATIONALITY', 'GROUP_OF_INDIVIDUALS', 'QUANTITY_FUZZY', 'RIOT', 'WEIGHT', 'THEFT', 'MILITARY', 'NON_GOVERNMENTAL_ORGANISATION', 'LATITUDE', 'COUP_D_ETAT', 'ELECTION', 'HOOLIGANISM_TROUBLEMAKING', 'QUANTITY_EXACT', 'AGITATING_TROUBLE_MAKING'}

generated_data_dir_path = os.path.join(INTERIM_DIR, "multilabel_tagged_text_dataset")
assert os.path.exists(generated_data_dir_path)
train_dir = os.path.join(generated_data_dir_path, "train")
test_dir = os.path.join(generated_data_dir_path, "test")

preprocessed_data_dir = os.path.join(INTERIM_DIR, "one_hot_multilabel_tagged_text_dataset")
labeled_preprocessed_data_dir_path = os.path.join(preprocessed_data_dir,"train")
! mkdir -p {labeled_preprocessed_data_dir_path}

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [3]:
def get_cat_var_distribution(cat_var: pd.Series) -> pd.DataFrame:
    return pd.concat(
        [cat_var.value_counts(), cat_var.value_counts(normalize=True)], axis=1
    )

## Prepare the datasets for the binary text classification

### Load and process the target

In [4]:
def load_csv(dir_or_file_path: str, index_col=None, sep=',') -> pd.DataFrame:
    if os.path.isdir(dir_or_file_path):
        all_files = glob.glob(os.path.join(dir_or_file_path , "*.csv"))  
    else:
        assert dir_or_file_path.endswith(".csv")
        all_files = [dir_or_file_path]
    assert len(all_files) > 0
    return pd.concat([pd.read_csv(filename, index_col=index_col, header=0, sep=sep) for filename in all_files], axis=0, ignore_index=True)

train_df = load_csv(train_dir).assign(**{TASK_TARGET_COL: lambda df: 2 + ~pd.isnull(df.relations).astype(int)}).drop(TARGET_COL, axis=1)
train_df

,text_index,e1,e2,text,label
0,2576,0,0,"Le matin du 10 janvier 2010, Arthur et Jacques...",0
1,2576,1,0,"Le matin du 10 janvier 2010, Arthur et Jacques...",0
2,2576,0,1,"Le matin du 10 janvier 2010, Arthur et Jacques...",1
3,2576,1,1,"Le matin du 10 janvier 2010, Arthur et Jacques...",0
4,2576,2,0,"Le matin du 10 janvier 2010, Arthur et Jacques...",0
...,...,...,...,...,...
124000,41884,11,22,"Le 14 janvier 2014, en pleine Sibérie, dans un...",0
124001,41884,12,22,"Le 14 janvier 2014, en pleine Sibérie, dans un...",0
124002,41884,15,22,"Le 14 janvier 2014, en pleine Sibérie, dans un...",0
124003,41884,17,22,"Le 14 janvier 2014, en pleine Sibérie, dans un...",1


In [5]:
get_cat_var_distribution(train_df[TASK_TARGET_COL])

,count,proportion
label,,
0,97611,0.787154
1,26394,0.212846


### train-validation split

In [6]:
VALIDATION_RATE = 0.2
train_df, val_df = train_test_split(train_df, test_size=VALIDATION_RATE, shuffle=True, random_state=RANDOM_SEED, stratify=train_df[TASK_TARGET_COL])
train_df.shape, val_df.shape

((99204, 5), (24801, 5))

In [7]:
get_cat_var_distribution(train_df[TASK_TARGET_COL])

,count,proportion
label,,
0,78089,0.787156
1,21115,0.212844


In [8]:
get_cat_var_distribution(val_df[TASK_TARGET_COL])

,count,proportion
label,,
0,19522,0.787146
1,5279,0.212854


### Undersampling the majority class

In [9]:
positive_train_df = train_df.query(f"{TASK_TARGET_COL}==1")
positive_train_df.shape

(21115, 5)

In [10]:
negative_train_df = train_df.query(f"{TASK_TARGET_COL}==0")
negative_train_df.shape

(78089, 5)

In [11]:
# balance classes
train_df = pd.concat([positive_train_df, negative_train_df.sample(positive_train_df.shape[0])], axis=0)
train_df.shape

(42230, 5)

## Create the tokenized datasets for model input

### init the tokenizer

In [12]:
# Hyperparameters
tokenizer = AutoTokenizer.from_pretrained(BASE_CHECKPOINT)
task_special_tokens = ["<e1>", "</e1>", "<e2>", "</e2>"] + [
    f"<{entity_class}>" for entity_class in entity_classes
]
# add special tokens to the tokenizer
num_added_tokens = tokenizer.add_tokens(task_special_tokens, special_tokens=True)
num_added_tokens, len(tokenizer)

/home/tagny/.pyenv/versions/3.11.8/envs/kgl11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


(59, 119606)

### init the train-valid datasets from dataframe

In [13]:
train_ds = Dataset.from_pandas(train_df, preserve_index=False)
val_ds = Dataset.from_pandas(val_df, preserve_index=False)
task_datasets = DatasetDict({"train": train_ds, "validation": val_ds})
task_datasets

DatasetDict({
    train: Dataset({
        features: ['text_index', 'e1', 'e2', 'text', 'label'],
        num_rows: 42230
    })
    validation: Dataset({
        features: ['text_index', 'e1', 'e2', 'text', 'label'],
        num_rows: 24801
    })
})

In [14]:
task_datasets["train"][0]

{'text_index': 51407,
 'e1': 0,
 'e2': 18,
 'text': "Le <e2><TIME_EXACT>12 mai 2016</e2>, un <e1><NATURAL_EVENT>tremblement de terre</e1> a frappé de plein fouet l'Arizona. Plusieurs personnes ont dû quitter leur domicile grâce à des camions et des hélicoptères mis à disposition par le gouvernement. Le <e1><NATURAL_EVENT>sinistre</e1> a provoqué la destruction de plusieurs bâtiments, dont des hôpitaux. Plusieurs patients ont dû être évacués en urgence. Parmi eux, il y avait Madame Belkiss Miller, une femme de 21 ans qui avait donné naissance à un bébé prématuré dans une ambulance à l'entrée de la ville de Phoenix. Les pompiers se sont servis d'un brancard et de serviettes qu'ils avaient sous la main pour aider la patiente. La jeune femme et son fils s'en sont sortis sains et saufs. Les opérations de sauvetage ont continué dans tout l'État.",
 'label': 1}

In [15]:
task_datasets["train"][1]

{'text_index': 11529,
 'e1': 0,
 'e2': 3,
 'text': 'Des centaines de manifestants, vêtus de t-shirts rouges, ont défilé hier matin en direction du Palais de l\'Élysée à Paris dans le cadre d\'une grève nationale. Le plus grand syndicat du pays souhaitait dénoncer la hausse des prix du carburant et les nombreuses coupures d\'électricité. Les manifestants ont bloqué les routes de la capitale en brûlant des pneus de voiture. Munis d\'une pancarte et d\'un <e2><MATERIEL>mégaphone</e2>, Madame <e1><CIVILIAN>Laura Chazal</e1>, porte-parole des manifestants et présidente de la fondation "Solidarité" à Paris, s\'est adressée au gouvernement. <e1><CIVILIAN>Elle</e1> a demandé au gouvernement d\'empêcher "un effondrement de l\'économie" et a dénoncé la hausse du coût de la vie.',
 'label': 1}

### Tokenize the datasets

In [16]:
def tokenize_function(example: dict):
    return tokenizer(example["text"], truncation=True)

# We’re using batched=True in our call to map so the function is applied to multiple elements of our dataset at once, and not on each element separately
# This is way faster
# Source https://huggingface.co/learn/nlp-course/chapter3/2?fw=pt
# columns are removed because DataCollatorWithPadding doesn't support any other columns than the ones produced by the tokenizer (or non tensors)
tokenized_datasets = task_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/42230 [00:00<?, ? examples/s]

Map:   0%|          | 0/24801 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text_index', 'e1', 'e2', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 42230
    })
    validation: Dataset({
        features: ['text_index', 'e1', 'e2', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 24801
    })
})

In [17]:
tokenized_datasets.column_names

{'train': ['text_index',
  'e1',
  'e2',
  'text',
  'label',
  'input_ids',
  'token_type_ids',
  'attention_mask'],
 'validation': ['text_index',
  'e1',
  'e2',
  'text',
  'label',
  'input_ids',
  'token_type_ids',
  'attention_mask']}

In [18]:
type(tokenized_datasets["train"][1]['attention_mask'])

list

### Test the batch-level padding with a data collator

In [43]:

samples = tokenized_datasets.remove_columns(task_datasets["train"].column_names)["train"][:8]
samples = {k: v for k, v in samples.items()}
[len(x) for x in samples["input_ids"]]

[196, 192, 216, 165, 238, 176, 188, 222]

In [61]:
task_datasets

DatasetDict({
    train: Dataset({
        features: ['text_index', 'e1', 'e2', 'text', 'label'],
        num_rows: 42230
    })
    validation: Dataset({
        features: ['text_index', 'e1', 'e2', 'text', 'label'],
        num_rows: 24801
    })
})

In [20]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 238]),
 'token_type_ids': torch.Size([8, 238]),
 'attention_mask': torch.Size([8, 238])}

## Fine-tuning a model with the Trainer API

### Compute the weight of classes to handle imbalance

In [21]:
# Source: https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#calculate_class_weights
# Scaling by total/2 helps keep the loss to a similar magnitude.
# n_examples = train_df.shape[0]
# n_classes = train_df[TASK_TARGET_COL].nunique()
# def compute_class_weights(lbl_df: pd.DataFrame) -> pd.Series:
#     return get_cat_var_distribution(lbl_df[TASK_TARGET_COL]).reset_index(drop=False)["count"].apply(lambda x: (1 / x) * (n_examples / n_classes)).rename("weight")
# pd.concat([get_cat_var_distribution(train_df[TASK_TARGET_COL]), compute_class_weights(train_df)], axis=1)

### Init the model

In [22]:
n_classes = train_df[TASK_TARGET_COL].nunique()
print(f"{n_classes=}")
model = AutoModelForSequenceClassification.from_pretrained(BASE_CHECKPOINT, num_labels=n_classes)
model.resize_token_embeddings(len(tokenizer))

n_classes=2


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(119606, 768)

### Init the trainer

Source: https://stackoverflow.com/questions/69087044/early-stopping-in-bert-trainer-instances#69087153

1. Use `load_best_model_at_end = True` (EarlyStoppingCallback() requires this to be True).
2. `evaluation_strategy = 'steps'` or IntervalStrategy.STEPS instead of 'epoch'.
3. `eval_steps = 50` (evaluate the metrics after N steps).
4. `metric_for_best_model = 'f1'`

In [23]:
def compute_metrics(p):    
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average="macro")
    precision = precision_score(y_true=labels, y_pred=pred, average="macro")
    f1 = f1_score(y_true=labels, y_pred=pred, average="macro")    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

training_args = TrainingArguments(
    output_dir=os.path.join(MODELS_DIR, f"{TASK_NAME}-byTrainerAPI-checkpoints"),
    per_device_train_batch_size=8,    
    per_device_eval_batch_size=32,
    num_train_epochs=20,
    evaluation_strategy=IntervalStrategy.EPOCH, # steps
    # eval_steps = 50, # Evaluation and Save happens every 50 steps
    learning_rate=2e-5,
    weight_decay=0.01,
    save_strategy=IntervalStrategy.EPOCH,
    logging_dir=os.path.join(MODELS_DIR, f"{TASK_NAME}-byTrainerAPI-tensorboard"),
    save_total_limit = 2, # Only last 2 models are saved. Older ones are deleted
    push_to_hub=False,
    metric_for_best_model = 'f1',
    load_best_model_at_end=True,
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)],
)

/home/tagny/.pyenv/versions/3.11.8/envs/kgl11/lib/python3.11/site-packages/accelerate/accelerator.py:447: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


### Launch the training

In [24]:
trainer.train()

[12:27:08|ERROR|jupyter.py:224] Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tagny (ifiafrik-team). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.427400,0.527162,0.794041,0.736185,0.829508,0.751284
2,0.362800,0.413547,0.858151,0.787394,0.845216,0.808832
3,0.366300,0.362972,0.838031,0.774930,0.869682,0.798185
4,0.331800,0.501049,0.834200,0.772531,0.869944,0.794984
5,0.300700,0.499140,0.865449,0.798501,0.875213,0.823925
6,0.299800,0.405769,0.879642,0.814105,0.877802,0.837930
7,0.255700,0.495491,0.875368,0.809180,0.878058,0.833918
8,0.231100,0.554259,0.860812,0.794899,0.879592,0.820819
9,0.185300,0.705833,0.853958,0.789351,0.880421,0.814861


TrainOutput(global_step=47511, training_loss=0.31548714157516866, metrics={'train_runtime': 9953.9804, 'train_samples_per_second': 84.85, 'train_steps_per_second': 10.607, 'total_flos': 4.884572362220724e+16, 'train_loss': 0.31548714157516866, 'epoch': 9.0})

### with pipeline